## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-21-21-41-00 +0000,world,Opinion | The Fallout From Syria’s Massacres,https://www.wsj.com/opinion/syria-sweida-massa...
1,2025-07-21-19-46-00 +0000,world,Opinion | Trump Arms Ukraine But Still Wants Out,https://www.wsj.com/opinion/trump-still-wants-...
2,2025-07-21-18-42-00 +0000,world,Opinion | Europe Will Never Keep Its Promises ...,https://www.wsj.com/opinion/europe-will-never-...
3,2025-07-21-14-01-00 +0000,world,Opinion | Toppling Tehran Isn’t Like Sacking S...,https://www.wsj.com/opinion/toppling-tehran-is...
4,2025-07-21-11-53-00 +0000,world,China Confirms Exit Ban on Wells Fargo Banker,https://www.wsj.com/world/china/china-confirms...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2289/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,2
7,europe,2
27,fight,2
48,japan,2
58,trade,2


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
6,2025-07-21-03-00-00 +0000,world,Germany Shut Down Thousands of Air-Raid Shelte...,https://www.wsj.com/world/europe/germany-new-a...,11
5,2025-07-21-03-00-00 +0000,world,"The Fiercest Border Fight: High Kicks, Mustach...",https://www.wsj.com/world/the-fiercest-border-...,11
8,2025-07-21-01-00-00 +0000,world,Why Israel’s Chaotic New Food Program in Gaza ...,https://www.wsj.com/world/middle-east/us-israe...,10
2,2025-07-21-18-42-00 +0000,world,Opinion | Europe Will Never Keep Its Promises ...,https://www.wsj.com/opinion/europe-will-never-...,8
10,2025-07-21-00-16-00 +0000,world,Japan Election Throws a Wrench in Trade Talks,https://www.wsj.com/world/asia/japan-election-...,8


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
6,11,2025-07-21-03-00-00 +0000,world,Germany Shut Down Thousands of Air-Raid Shelte...,https://www.wsj.com/world/europe/germany-new-a...
5,11,2025-07-21-03-00-00 +0000,world,"The Fiercest Border Fight: High Kicks, Mustach...",https://www.wsj.com/world/the-fiercest-border-...
2,8,2025-07-21-18-42-00 +0000,world,Opinion | Europe Will Never Keep Its Promises ...,https://www.wsj.com/opinion/europe-will-never-...
10,8,2025-07-21-00-16-00 +0000,world,Japan Election Throws a Wrench in Trade Talks,https://www.wsj.com/world/asia/japan-election-...
8,8,2025-07-21-01-00-00 +0000,world,Why Israel’s Chaotic New Food Program in Gaza ...,https://www.wsj.com/world/middle-east/us-israe...
4,7,2025-07-21-11-53-00 +0000,world,China Confirms Exit Ban on Wells Fargo Banker,https://www.wsj.com/world/china/china-confirms...
3,6,2025-07-21-14-01-00 +0000,world,Opinion | Toppling Tehran Isn’t Like Sacking S...,https://www.wsj.com/opinion/toppling-tehran-is...
0,3,2025-07-21-21-41-00 +0000,world,Opinion | The Fallout From Syria’s Massacres,https://www.wsj.com/opinion/syria-sweida-massa...
1,3,2025-07-21-19-46-00 +0000,world,Opinion | Trump Arms Ukraine But Still Wants Out,https://www.wsj.com/opinion/trump-still-wants-...
7,3,2025-07-21-02-01-00 +0000,world,Opinion | The Populist Wave Hits Japan,https://www.wsj.com/opinion/the-populist-wave-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
